In [1]:
import numpy as np
seed = 42069
np.random.seed(seed)
import pandas as pd
from matplotlib import pyplot as plt
import arviz
import pystan
from scipy import sparse, stats
from typing import Iterable, Union, Callable
from sklearn.model_selection import train_test_split, ParameterGrid
import altair as alt
from time import time, sleep
from tqdm import tqdm

# Own files
import utils 
import StanClasses

In [13]:
A = {'A':[1,2,3], 'B':[1,2,3], 'C':[1,2,3]}

pd.DataFrame(A)

,A,B,C
0,1,1,1
1,2,2,2
2,3,3,3


In [11]:
from multiprocessing import Pool

def bigboi(a,b,c,d):
    sleep(2)
    return a, b, c, d
    
args = [
    ('NMF','fit_time1','train_mae','val_mae'),
    ('NMF','fit_time2','train_mae','val_mae'),
    ('ARD','fit_time3','train_mae','val_mae'),
    ('ARD','fit_time4','train_mae','val_mae')
]
    
with Pool(None) as p:
    a = p.starmap(bigboi, args)
    print(a)
    

[('NMF', 'fit_time1', 'train_mae', 'val_mae'), ('NMF', 'fit_time2', 'train_mae', 'val_mae'), ('ARD', 'fit_time3', 'train_mae', 'val_mae'), ('ARD', 'fit_time4', 'train_mae', 'val_mae')]
